In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt
from math import ceil, floor
from datetime import datetime 
from dataclasses import dataclass
from torchvision import datasets, transforms
from torch.utils.data import Dataset, TensorDataset, DataLoader
from src.models.localization.cnn_localizer import CNNLocalizer
from src.util.loss_funcs import localization_loss
import seaborn as sns
from src.util.transform_dataset import TransformDataset, get_transform

# Detection

In [47]:
torch.manual_seed(123)
torch.set_default_dtype(torch.float32) # TODO maybe remove
batch_size = 128       
torch.set_printoptions(profile="full")     

### Load data and preprocessing

In [48]:
H_in, W_in = 48, 60
H_out, W_out = 2, 3
CELL_WIDTH, CELL_HEIGHT = W_in / W_out, H_in / H_out

def get_cell(x, y):
    row = (y * H_in) // (CELL_HEIGHT)
    col = (x * W_in) // (CELL_WIDTH)
    return int(row), int(col)

def convert_Y_label(Y:torch.Tensor):
    converted_Y = [[[0,0,0,0,0,0] for _ in range(W_out)] for _ in range(H_out)]

    for digit in Y:
        p, x, y, w, h, c = digit
        row, col = get_cell(x.item(), y.item())

        x = ((x * W_in) - col * CELL_WIDTH) / (CELL_WIDTH)
        y = ((y * H_in) - row * CELL_HEIGHT) / (CELL_HEIGHT)
        w *= W_out
        h *= H_out
         
        converted_Y[row][col] = [p, x, y, w, h, c]

    return torch.Tensor(converted_Y)

In [ ]:
train_true = torch.load("data/list_y_true_train.pt")
val_true = torch.load("data/list_y_true_val.pt")
test_true = torch.load("data/list_y_true_test.pt")

train_images = torch.load("data/detection_train.pt").tensors[0]
val_images = torch.load("data/detection_val.pt").tensors[0]
test_images = torch.load("data/detection_test.pt").tensors[0]


converted_data = [torch.zeros(N, H_out, W_out, 6) for N in [len(train_true), len(val_true), len(test_true)]]
for i, dataset in enumerate([train_true, val_true, test_true]):
    for j in range(len(dataset)):
        converted_data[i][j] = convert_Y_label(dataset[j])

train_labels, val_labels, test_labels = converted_data

transforms = get_transform(train_images)

train_loader = DataLoader(TransformDataset(TensorDataset(train_images, train_labels), transforms), batch_size=batch_size, shuffle=False)
val_loader = DataLoader(TransformDataset(TensorDataset(val_images, val_labels), transforms), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TransformDataset(TensorDataset(test_images, test_labels), transforms), batch_size=batch_size, shuffle=False)


C:\Users\krist\AppData\Local\Temp\ipykernel_6840\1809610471.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_true = torch.load("data/list_y_true_train.pt")
C:\Users

### Training

### Prediction

In [52]:
# TODO

### Model selection and evaluation

In [53]:
def compute_IoU_detection(model, loader, preprocessor):
    """
    Compute IoU performance of the model on the given dataset
    """
    # TODO

def compute_accuracy_detection(model, loader, preprocessor):
    """
    Compute accuracy performance of the model on the given dataset
    """
   # tODO